In [1]:
from cleanfid import fid
import numpy as np
import os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch
from torchvision import transforms as T,datasets,models

c:\Users\Eachann\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 24

In [3]:
result_path = "experiments/"+ "orig_test" + "/results"

In [4]:
from ignite.metrics import FID, InceptionScore
from ignite.engine import Engine
real = result_path#+"/hr"
fake = result_path+"/sr"
trainset = datasets.ImageFolder(real,transform=T.ToTensor())
train_loader = DataLoader(dataset=trainset, batch_size=batch_size)
images,labels = iter(train_loader).next()
print(len(images),labels)
print(images[12].shape)
plt.imshow(torch.einsum('abc->bca',images[2]))

RuntimeError: stack expects each tensor to be equal size, but got [3, 251, 257] at entry 0 and [3, 128, 128] at entry 2

In [ ]:
def interpolate(batch):
    arr = []
    for img in batch:
        pil_img = T.ToPILImage()(img)
        resized_img = pil_img.resize((299,299), Image.BILINEAR)
        arr.append(T.ToTensor()(resized_img))
    return torch.stack(arr)
with torch.no_grad():
    real = torch.einsum('qabc->qbca',images[:11,:,:,:])
    fake = torch.einsum('qabc->qbca',images[11:,:,:,:])
    #fake = interpolate(fake.cpu())
    assert real.shape==fake.shape
    print(real.shape)
print(fake.shape)   


torch.Size([11, 128, 128, 3])
torch.Size([11, 128, 128, 3])


In [ ]:

def evaluation_step(engine, batch):
    with torch.no_grad():
        fake = fake.view(batch_size, 28, 28)
        #print(fake.shape,"*"*30)
        fake = interpolate(fake.cpu())
        #print(fake.shape,"%"*30)
        real = interpolate(batch[0])
    return fake, real
fid_metric = FID(device="cuda")
is_metric = InceptionScore(device="cuda", output_transform=lambda x: x[0])
evaluator = Engine(evaluation_step)
fid_metric.attach(evaluator, "fid")
is_metric.attach(evaluator, "is")
evaluator.run(train_loader, max_epochs=24) # use your test data loader, NOT training data loader
metrics = evaluator.state.metrics
fid_score = metrics['fid']
is_score = metrics['is']

Current run is terminating due to exception: local variable 'fake' referenced before assignment
Engine run is terminating due to exception: local variable 'fake' referenced before assignment


UnboundLocalError: local variable 'fake' referenced before assignment

In [ ]:
result_path = "experiments/orig_test/results"
#print(os.listdir(result_path+"/hr"))
avg_fid =  fid.compute_fid(result_path+"/1",result_path+"/2", num_workers=0)
print(avg_fid)

compute FID between two folders
Found 2 images in the folder experiments/orig_test/results/1


FID 1 : 100%|██████████| 1/1 [00:00<00:00,  3.96it/s]


Found 2 images in the folder experiments/orig_test/results/2


FID 2 : 100%|██████████| 1/1 [00:06<00:00,  6.66s/it]


215.15823364257812
